# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [96]:
# import libraries
import pandas as pd

In [97]:
# load data from database
from sqlalchemy import create_engine
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table('message', con=engine)
X = df.iloc[:,4:]
Y = df['message']


In [98]:
X.columns

Index(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'child_alone', 'water', 'food', 'shelter', 'clothing', 'money',
       'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object')

### 2. Write a tokenization function to process your text data

In [99]:
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
nltk.download('stopwords')
nltk.download('punkt')
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer, PorterStemmer
stopwords = list(nltk.corpus.stopwords.words('english'))

def tokenize(text):
    clean_Y = []
    for sentence in text:
        sentence = re.sub('[^a-zA-Z0-9]', ' ', sentence )
        sent_tok = word_tokenize(sentence)
        clean = []
        for i in sent_tok:
            if i not in stopwords:
                clean.append(i.lower())
        
#         print(' '.join(clean))
        clean_Y.append(clean)
    
    return clean_Y
Y = tokenize(Y)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [100]:
Y

[['weather', 'update', 'cold', 'front', 'cuba', 'could', 'pass', 'haiti'],
 ['is', 'hurricane'],
 ['looking', 'someone', 'name'],
 ['un',
  'reports',
  'leogane',
  '80',
  '90',
  'destroyed',
  'only',
  'hospital',
  'st',
  'croix',
  'functioning',
  'needs',
  'supplies',
  'desperately'],
 ['says', 'west', 'side', 'haiti', 'rest', 'country', 'today', 'tonight'],
 ['information', 'national', 'palace'],
 ['storm', 'sacred', 'heart', 'jesus'],
 ['please', 'need', 'tents', 'water', 'we', 'silo', 'thank'],
 ['i', 'would', 'like', 'receive', 'messages', 'thank'],
 ['i',
  'croix',
  'des',
  'bouquets',
  'we',
  'health',
  'issues',
  'they',
  'workers',
  'santo',
  '15',
  'area',
  'croix',
  'des',
  'bouquets'],
 ['there', 'nothing', 'eat', 'water', 'starving', 'thirsty'],
 ['i', 'petionville', 'i', 'need', 'information', 'regarding', '4636'],
 ['i',
  'thomassin',
  'number',
  '32',
  'area',
  'named',
  'pyron',
  'i',
  'would',
  'like',
  'water',
  'thank',
  'god',
 

In [101]:
tfidf = TfidfVectorizer(preprocessor=lambda x: x, tokenizer=lambda x: x)
tfidf_matrix = tfidf.fit_transform(Y)

In [102]:
tfidf_matrix

<26180x35066 sparse matrix of type '<class 'numpy.float64'>'
	with 388120 stored elements in Compressed Sparse Row format>

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [103]:
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression


pipeline = MultiOutputClassifier(LogisticRegression())

In [104]:
# for column in X:
#     print(column, X[column].unique())
    
X = X.drop('child_alone', axis =1)

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [105]:
X_train, X_test, y_train, y_test = train_test_split(X, tfidf_matrix, test_size=0.4)
pipeline.fit(y_train, X_train)

MultiOutputClassifier(estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
           n_jobs=1)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [106]:
X_pred = pipeline.predict(y_test)

In [11]:
import numpy as np
print(np.shape(X_pred))
X_test.shape

(10472, 35)


(10472, 35)

In [12]:
X_pred_df = pd.DataFrame(X_pred)

In [13]:
X_pred_df.columns = X_test.columns

In [14]:
X_pred_df.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,water,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,1,0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
from sklearn.metrics import classification_report
target_names = X.columns
for column in X_pred_df:
    print(column)
    print(classification_report(X_test[column], X_pred_df[column]))

related
             precision    recall  f1-score   support

          0       0.77      0.35      0.48      2485
          1       0.82      0.97      0.89      7910
          2       0.00      0.00      0.00        77

avg / total       0.80      0.81      0.78     10472

request
             precision    recall  f1-score   support

          0       0.90      0.98      0.94      8686
          1       0.83      0.50      0.62      1786

avg / total       0.89      0.90      0.89     10472

offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00     10428
          1       0.00      0.00      0.00        44

avg / total       0.99      1.00      0.99     10472

aid_related
             precision    recall  f1-score   support

          0       0.78      0.87      0.82      6209
          1       0.78      0.63      0.70      4263

avg / total       0.78      0.78      0.77     10472

medical_help
             precision    recall  f1-sco

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [16]:
from sklearn.model_selection import GridSearchCV
parameters = {'n_jobs':[1,2,3,4]}

cv = GridSearchCV(pipeline, parameters, cv=5)
cv.fit(y_train, X_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=MultiOutputClassifier(estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
           n_jobs=1),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'n_jobs': [1, 2, 3, 4]}, pre_dispatch='2*n_jobs',
       refit=True, return_train_score='warn', scoring=None, verbose=0)

In [17]:
cv.best_estimator_

MultiOutputClassifier(estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
           n_jobs=1)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [18]:
pipeline = MultiOutputClassifier(estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
           n_jobs=1)

pipeline.fit(y_train, X_train)
X_pred = pipeline.predict(y_test)
X_pred_df_imp = pd.DataFrame(X_pred)
X_pred_df_imp.columns = X_test.columns

for column in X_pred_df_imp:
    print(column)
    print(classification_report(X_test[column], X_pred_df_imp[column]))

related
             precision    recall  f1-score   support

          0       0.77      0.35      0.48      2485
          1       0.82      0.97      0.89      7910
          2       0.00      0.00      0.00        77

avg / total       0.80      0.81      0.78     10472

request
             precision    recall  f1-score   support

          0       0.90      0.98      0.94      8686
          1       0.83      0.50      0.62      1786

avg / total       0.89      0.90      0.89     10472

offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00     10428
          1       0.00      0.00      0.00        44

avg / total       0.99      1.00      0.99     10472

aid_related
             precision    recall  f1-score   support

          0       0.78      0.87      0.82      6209
          1       0.78      0.63      0.70      4263

avg / total       0.78      0.78      0.77     10472

medical_help
             precision    recall  f1-sco

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [19]:
from sklearn.ensemble import RandomForestClassifier
pipeline = MultiOutputClassifier(estimator=RandomForestClassifier(), n_jobs=1)

pipeline.fit(y_train, X_train)
X_pred = pipeline.predict(y_test)
X_pred_df_imp = pd.DataFrame(X_pred)
X_pred_df_imp.columns = X_test.columns

for column in X_pred_df_imp:
    print(column)
    print(classification_report(X_test[column], X_pred_df_imp[column]))

related
             precision    recall  f1-score   support

          0       0.64      0.40      0.49      2485
          1       0.83      0.92      0.87      7910
          2       0.30      0.42      0.35        77

avg / total       0.78      0.80      0.78     10472

request
             precision    recall  f1-score   support

          0       0.89      0.98      0.93      8686
          1       0.80      0.41      0.54      1786

avg / total       0.87      0.88      0.86     10472

offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00     10428
          1       0.00      0.00      0.00        44

avg / total       0.99      1.00      0.99     10472

aid_related
             precision    recall  f1-score   support

          0       0.75      0.86      0.80      6209
          1       0.74      0.58      0.65      4263

avg / total       0.75      0.75      0.74     10472

medical_help
             precision    recall  f1-sco

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 9. Export your model as a pickle file

In [20]:
import pickle
pickle.dump(cv, open('model.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.